<a href="https://www.kaggle.com/code/sivamanivallabhani/bertfinetuning?scriptVersionId=175679587" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# Regular imports
import matplotlib.pyplot as plt
import torch
from torch import nn
import numpy as np 
device = "cuda" if torch.cuda.is_available() else "cpu"

# Import torch.nn.functional as F
import torch.nn.functional as F

In [2]:
# Define data path
data_path = "/kaggle/input/davidson1/labeled_data.csv"

In [3]:
# Load data
import pandas as pd
data = pd.read_csv(data_path)
df = pd.DataFrame(data)
df.columns = ['Unnamed: 0', 'count', 'hate_speech', 'offensive_language', 'neither',
              'classk', 'tweet']
df["classk"] = df["classk"].map({2:1, 1:0, 0:0})  # Convert labels to binary (hate vs not hate)
tweets = df.tweet
res = []


In [4]:
# Process tweets
for i in range(len(tweets)):
    splitt = tweets[i].split(":")
    if len(splitt) == 1:
        res.append(splitt[0])
    else:
        res.append(splitt[-1])

tweets = res

In [5]:

# Import tokenizer and BERT model
from transformers import AutoTokenizer, BertModel
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [6]:
# Define custom dataset class
class Custom_Dataset(torch.utils.data.Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        
    def __getitem__(self, index: int) -> (torch.Tensor, int):
        X = self.data[index]
        y = self.labels[index]
        return (X, y)
    
    def __len__(self) -> int:
        return len(self.data)


In [7]:
# Split data into train and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(tweets, list(df.classk), test_size=0.2, shuffle=True)

# Define train and test datasets and dataloaders
train_data = Custom_Dataset(data=X_train, labels=y_train)
test_data = Custom_Dataset(data=X_test, labels=y_test)
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=3, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_data, batch_size=1, shuffle=False)

In [8]:
class Model0(nn.Module):
    def __init__(self, model, tokenizer):
        super().__init__()
        self.tokenizer = tokenizer
        self.model = model
        self.fc1 = nn.Linear(768, 256)  # Additional layer for fine-tuning
        self.fc2 = nn.Linear(256, 2)     # Output layer

    def forward(self, X):
        inputs = self.tokenizer(X, return_tensors="pt").to(device)
        outputs = self.model(**inputs)
        outputs = outputs.last_hidden_state[0][0].unsqueeze(1)

        
        # Additional layers for fine-tuning
        outputs = torch.relu(self.fc1(outputs.squeeze()))  # Remove extra dimension
        outputs = self.fc2(outputs)

        return outputs


In [9]:
# Instantiate the model
model0 = Model0(model, tokenizer)
model0.to(device)

# Define the optimizer
optimizer = torch.optim.Adam(params=model0.parameters(), lr=0.00002)

In [10]:
# Define the inter and intra space loss functions
def L_inter(spaces):
    means = [torch.mean(spaces[i]) for i in range(spaces.shape[0])]
    loss = torch.tensor(0.0, requires_grad=True)
    for k in range(len(means)):
        cur = torch.tensor(0.0, requires_grad=True)
        for l in range(len(means)):
            if l == k:
                continue
            cur = cur + (1 / (1 - ((means[k] * means[l]))))
        loss = loss + cur
    return loss

def L_intra(spaces):
    loss = torch.tensor(0.0, requires_grad=True)
    for k in range(len(spaces)):
        loss = loss + 1 / torch.var(spaces[k], dim=0)
    return torch.sum(loss) / loss.shape[0]

In [11]:
# Evaluate the model
correct = 0
total = len(y_test)

true = []
preds = []

with torch.no_grad():
    for i, batch in enumerate(test_loader):
        if i == 100 :
            break
        X, y = batch
        y_preds = model0(X)

        # Move y tensor to the same device as y_preds
        y = y.to(y_preds.device)

        y_predict = torch.argmax(y_preds)

        true.extend(y.cpu())  # Move true labels back to CPU
        preds.append(y_predict.cpu())  # Move predictions back to CPU

        cur = torch.eq(y_predict, y)

        correct += torch.sum(cur)

print(f"Test Accuracy: {correct/total}")


Test Accuracy: 0.008876336738467216


In [12]:
# Define hate and not_hate words
hate_words = ['Moist', 'Cunt', 'Panties', 'Fuck', 'Hate', 'Nigger', 'Pussy', 'Ass', 
              'Motherfucker', 'Bitch', 'Damn']
not_hate_words = ['Love', 'Peace', 'Kindness', 'Happiness', 'Respect', 'Friendship',
                  'Appreciation', 'Hope', 'Encouragement', 'Support', 'Caring']

In [13]:
import torch
def cosine_similarity(p, q, eps=1e-8):

    p = torch.clamp(p, min=eps)
    q = torch.clamp(q, min=eps)

    # Normalize vectors (assuming L2 norm for single-dimensional case)
    norm_p = torch.linalg.norm(p)
    norm_q = torch.linalg.norm(q)
    p = p / (norm_p + eps)  # Avoid division by zero
    q = q / (norm_q + eps)

    # Calculate cosine similarity
    return torch.dot(p, q)  # Dot product



def ContrastiveLoss():

    loss = torch.tensor(0.0).to(device)  # Move loss to device and set to zero initially

  # Process hate words
    hate_embeddings = []
    for word in hate_words:
        inputs = tokenizer(word, return_tensors="pt").to(device)
        with torch.no_grad():  # Disable gradient calculation for efficiency
            outputs = model(**inputs)
        hate_embeddings.append(torch.mean(outputs.last_hidden_state[0][1:], dim=0))  # Extract and average word embedding

    hate_space = torch.stack(hate_embeddings).requires_grad_().to(device)  # Create and enable gradients for hate embeddings

    # Process not-hate words
    not_hate_embeddings = []
    for word in not_hate_words:
        inputs = tokenizer(word, return_tensors="pt").to(device)
        with torch.no_grad():  # Disable gradient calculation for efficiency
            outputs = model(**inputs)
        not_hate_embeddings.append(torch.mean(outputs.last_hidden_state[0][1:], dim=0))

    not_hate_space = torch.stack(not_hate_embeddings).requires_grad_().to(device)  # Create and enable gradients for not-hate embeddings

    # Calculate loss terms
    for i in hate_space:
        for j in not_hate_space:
          
            loss += cosine_similarity(i, j)   

    for i in hate_space:
        for j in hate_space:
            loss -= 0.4*cosine_similarity(i, j)   

    for i in not_hate_space:
        for j in not_hate_space:
            loss -= 0.4*cosine_similarity(i, j) 

    return loss

In [14]:
# Train the model
model0.train()
try:
    for _ in range(5):
        for i, batch in enumerate(train_loader):
#             if i==100:
#                 break
            if i % 50 == 0:
                print(i)

            X, Y = batch
            loss = 0

            for X, y in zip(X, Y):
                y_preds = model0(X)
                loss += F.cross_entropy(y_preds.view(1, -1).to(device), torch.tensor([y]).to(device))

#             loss /= len(Y)
#             loss += 0.7 * L_inter(torch.cat((hate_space, not_hate_space))) + 0.5 * L_intra(torch.stack([hate_space, not_hate_space]))
            loss+=5*ContrastiveLoss()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        

except KeyboardInterrupt:
    print("Training interrupted by the user.")

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900
3950
4000
4050
4100
4150
4200
4250
4300
4350
4400
4450
4500
4550
4600
4650
4700
4750
4800
4850
4900
4950
5000
5050
5100
5150
5200
5250
5300
5350
5400
5450
5500
5550
5600
5650
5700
5750
5800
5850
5900
5950
6000
6050
6100
6150
6200
6250
6300
6350
6400
6450
6500
6550
6600
0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3

In [15]:
# Evaluate the model
correct = 0
total = len(y_test)

true = []
preds = []

with torch.no_grad():
    for i, batch in enumerate(test_loader):
        if i == 100 :
            break
        X, y = batch
        y_preds = model0(X)

        # Move y tensor to the same device as y_preds
        y = y.to(y_preds.device)

        y_predict = torch.argmax(y_preds)

        true.extend(y.cpu())  # Move true labels back to CPU
        preds.append(y_predict.cpu())  # Move predictions back to CPU

        cur = torch.eq(y_predict, y)

        correct += torch.sum(cur)

print(f"Test Accuracy: {correct/total}")


Test Accuracy: 0.018559612333774567


In [16]:
import torch.nn.functional as F

# Example usage:
predicted_probs = torch.tensor([[0.9, 0.1], [0.3, 0.7], [0.8, 0.2]])
target_labels = torch.tensor([0, 1, 0])  # Assuming binary classification
ce_loss = F.cross_entropy(predicted_probs, target_labels)
ce_loss

tensor(0.4405)

In [17]:
temp = []
for word in hate_words:
    inputs = tokenizer(word, return_tensors="pt").to(device)
    outputs = model(**inputs)
    outputs = torch.mean(outputs.last_hidden_state[0][1:], dim=0)
    temp.append(outputs.tolist())

hate_space = torch.tensor(temp).to(device).requires_grad_()

# Process not_hate words
temp = []
for word in not_hate_words:
    inputs = tokenizer(word, return_tensors="pt").to(device)
    outputs = model(**inputs)
    outputs = torch.mean(outputs.last_hidden_state[0][1:], dim=0)
    temp.append(outputs.tolist())

not_hate_space = torch.tensor(temp).to(device).requires_grad_()

In [18]:
import random
l=[[random.uniform(0,1)] for i in range(11)]
l

[[0.41060029860791747],
 [0.09599322766664087],
 [0.6738940013049377],
 [0.605164743133084],
 [0.7805677351638933],
 [0.9246160426841356],
 [0.9987275345667709],
 [0.3605233626496407],
 [0.5433614118104958],
 [0.9824158083752524],
 [0.19374924974617103]]

In [19]:
hate_space.max(1)[0]

tensor([1.1789, 1.9911, 1.9859, 2.0179, 1.9384, 1.6565, 1.9924, 2.1631, 1.9721,
        1.9269, 2.1648], device='cuda:0', grad_fn=<MaxBackward0>)

In [20]:
# next(iter(test_loader)) 
tensor_shape = (11,1)
hate_space_weights = torch.tensor(l).to(device).requires_grad_()
not_hate_space_weights = torch.tensor(l).to(device).requires_grad_()

print(hate_space_weights.device,not_hate_space_weights.is_leaf)

cuda:0 True


In [21]:
hate_space_weights.shape

torch.Size([11, 1])

In [22]:
hate_space_weights.is_leaf

True

In [23]:
import torch.nn.functional as F

class Model1(nn.Module):

    def __init__(self, model, tokenizer):
        super().__init__()
        self.tokenizer = tokenizer
        self.model = model

    def forward(self, X):
        inputs = self.tokenizer(X, return_tensors="pt").to(device)
        outputs = self.model(**inputs)
        outputs = outputs.last_hidden_state[0][1:len(outputs.last_hidden_state[0])].transpose(0, 1)#shape=>(768,no of tokens)
        col_norms = torch.norm(outputs, p=2, dim=0, keepdim=True)
        outputs = outputs / col_norms#  makes every word resp unit vector 
        hate_att_scores=torch.matmul(hate_space/torch.norm(hate_space,p=2, dim=0, keepdim=True),outputs)#shape=>(11,v)
        not_hate_att_scores=torch.matmul(not_hate_space/torch.norm(not_hate_space,p=2, dim=0, keepdim=True),outputs)#shape=>(11,v)
        Psk_hate = torch.matmul(hate_att_scores.max(1)[0], hate_space_weights)#shape=>(1)
        Psk_not_hate = torch.matmul(not_hate_att_scores.max(1)[0], not_hate_space_weights)#shape=>(1)
        output_tensor = torch.cat((Psk_hate, Psk_not_hate), dim=0).requires_grad_()#
        return output_tensor

In [24]:
model1 = Model1(model, tokenizer)

In [25]:
def L_inter(spaces):
    """this function calculates inter space losses between spaces 
    forces the class spaces of different class to move apart
    """
    means = [torch.mean(spaces[i]) for i in range(spaces.shape[0])]
    
    loss = torch.tensor(0.0, requires_grad=True)
    
    for k in range(len(means)):
        cur = torch.tensor(0.0, requires_grad=True)
        for l in range(len(means)):
            if l == k:
                continue
            cur = cur + (1 / (1 - ((means[k] * means[l]))))
        
        loss = loss + cur
    
    return loss
            
            

def L_intra(spaces):
    """"this function calculates intra space loss 
forces the vector representations inside a class space to have high variability(varience)
"""
    loss=torch.tensor(0.0, requires_grad=True)
    for k in range(len(spaces)):
        loss=loss+1/torch.var(spaces[k], dim=0)
    
    return torch.sum(loss)/loss.shape[0]

In [26]:
print(hate_space,not_hate_space)

tensor([[-7.6569e-01,  2.1203e-01,  5.0374e-01,  ...,  3.3824e-01,
          5.6870e-02, -8.8775e-03],
        [ 8.2112e-02,  7.7441e-01, -2.8465e-01,  ..., -7.0126e-02,
          2.7464e-01, -8.5519e-01],
        [-2.0059e-01,  1.2401e-01,  6.2270e-01,  ...,  5.5178e-02,
          3.7917e-01, -4.2764e-01],
        ...,
        [-9.4384e-03,  9.7978e-01, -6.0234e-02,  ..., -7.5647e-02,
          4.3120e-01, -1.1975e+00],
        [-3.2130e-02,  8.5117e-01, -2.8864e-01,  ..., -2.1999e-01,
          2.6469e-01, -1.1723e+00],
        [-1.1273e-03,  9.5339e-01, -5.1896e-02,  ...,  5.4020e-02,
          4.6106e-01, -1.1064e+00]], device='cuda:0', requires_grad=True) tensor([[ 0.1840,  0.3677,  0.3408,  ...,  0.2126,  0.0806, -0.1744],
        [-0.0538,  0.2244,  0.2598,  ...,  0.3274,  0.3125, -0.5707],
        [-0.3049,  0.1907,  0.5708,  ...,  0.0714, -0.0292,  0.0945],
        ...,
        [-0.3566,  0.2331,  0.5323,  ...,  0.0845, -0.0718, -0.0949],
        [-0.0196,  0.2423,  0.2362,  .

In [27]:
print(hate_space_weights,not_hate_space_weights)

tensor([[0.4106],
        [0.0960],
        [0.6739],
        [0.6052],
        [0.7806],
        [0.9246],
        [0.9987],
        [0.3605],
        [0.5434],
        [0.9824],
        [0.1937]], device='cuda:0', requires_grad=True) tensor([[0.4106],
        [0.0960],
        [0.6739],
        [0.6052],
        [0.7806],
        [0.9246],
        [0.9987],
        [0.3605],
        [0.5434],
        [0.9824],
        [0.1937]], device='cuda:0', requires_grad=True)


In [28]:
from sklearn.metrics import precision_score, recall_score,accuracy_score


model1.to(device)

print(hate_space.is_leaf)
optimizer = torch.optim.Adam(params=[hate_space, not_hate_space, hate_space_weights, not_hate_space_weights], lr=0.0002)
model1.train()


curr=0.1
prev=0
pprev=0
for _ in range(5):
    for i, batch in enumerate(train_loader):
#         if i == 100:
#             break
        if i % 2000 == 0:
            print(i)

        x, Y = batch
        loss = 0

        for X, y in zip(x, Y):
            probs = model1(X)
#             print(y_preds)
            #print(f"printing y_preds={y_preds} shape={y_preds.shape}")

            loss+=F.cross_entropy(probs.view(1, -1).to(device), torch.tensor([y]).to(device))
            """explisit soft max is not used to convert to probs is cross 
            entrophy loss has inbuilt softmax  function adding softmax explisity
            will effect adversly by smoothing again on smoothed values will decrese 
            the weightage for class with high prob """
#             break
        loss /= len(x)
        loss += 1*L_inter(torch.cat((hate_space, not_hate_space))) + 1*L_intra(torch.stack([hate_space, not_hate_space]))
            
        optimizer.zero_grad()
        
        loss.backward()
        optimizer.step()
#         print(hate_space.grad,hate_space_weights.grad)
    correct = 0
    total = len(y_test)

    true = []
    preds = []

    with torch.inference_mode():
        for i, batch in enumerate(test_loader):
#             if i==100:
#                 break

            X, y = batch
            Psk_hate, Psk_not_hate = model1(X)

            y_predict = torch.argmax(torch.tensor([Psk_hate.mean().item(), Psk_not_hate.mean().item()]))

            true.extend(y)
            preds.append(y_predict)

            cur = torch.eq(y_predict, y)

            correct += torch.sum(cur)
    pprev=prev
    prev=curr
    print(f"Test Accuracy: {correct/total}")
    curr=correct/total
    print(f"Accuracy - {accuracy_score(y_test, preds)}")
    print(f"precison score -{precision_score(y_test, preds, average='macro')}")
    print(f"recall score  -{recall_score(y_test, preds, average='macro')}")
  

True
0
2000
4000
6000
Test Accuracy: 0.9221303462982178
Accuracy - 0.9221303207585233
precison score -0.8679253250664789
recall score  -0.8455910188858421
0
2000
4000
6000
Test Accuracy: 0.9213233590126038
Accuracy - 0.9213233810772645
precison score -0.8666535458797098
recall score  -0.8436647819906402
0
2000
4000
6000
Test Accuracy: 0.9271736741065979
Accuracy - 0.9271736937663909
precison score -0.8864017914495518
recall score  -0.8414130578708119
0
2000
4000
6000
Test Accuracy: 0.9251563549041748
Accuracy - 0.9251563445632439
precison score -0.8764733380873148
recall score  -0.8464477560922401
0
2000
4000
6000
Test Accuracy: 0.9249545931816101
Accuracy - 0.9249546096429292
precison score -0.8762318389030748
recall score  -0.8458460713750319


In [29]:
print(hate_space_weights,not_hate_space_weights)

tensor([[ 0.5067],
        [-0.1763],
        [ 0.6888],
        [ 0.3148],
        [ 0.7838],
        [ 0.6165],
        [ 0.7928],
        [ 0.1339],
        [ 0.4366],
        [ 0.8223],
        [ 0.1977]], device='cuda:0', requires_grad=True) tensor([[0.3641],
        [0.1183],
        [0.4038],
        [0.6030],
        [0.6723],
        [0.6732],
        [0.9079],
        [0.2568],
        [0.3354],
        [0.6541],
        [0.1620]], device='cuda:0', requires_grad=True)


In [30]:
print(hate_space,not_hate_space)

tensor([[-2.4926, -1.5655,  2.7308,  ...,  1.8629, -0.6847,  1.8373],
        [ 3.4731,  3.0875, -3.4687,  ..., -4.2849, -3.4961,  3.3900],
        [-1.4124, -1.3522,  2.5527,  ...,  1.7379,  2.1016,  0.6787],
        ...,
        [ 0.9566,  3.2839,  0.4615,  ..., -2.4343,  2.3699, -3.1645],
        [-0.4243,  2.4735, -1.1650,  ..., -1.3173, -0.8284, -2.4734],
        [ 2.2015,  3.3421, -1.5935,  ...,  3.3182,  4.0223, -3.2550]],
       device='cuda:0', requires_grad=True) tensor([[ 2.2183,  3.5233, -2.0656,  ...,  3.2314,  3.2166, -1.9926],
        [ 2.4009, -3.2613, -3.1266,  ...,  3.5531,  2.7573, -3.5552],
        [-2.6057, -1.4621,  2.5893,  ..., -2.7032, -0.7434,  2.4119],
        ...,
        [-2.8658, -2.2081,  3.6804,  ..., -3.6331, -1.4868, -2.1869],
        [ 1.4463, -1.5292, -0.8523,  ...,  2.6485, -0.6623,  1.2459],
        [ 2.2917, -2.3755,  4.3250,  ...,  3.2445, -3.8571, -3.2154]],
       device='cuda:0', requires_grad=True)


In [31]:
def save_bert_model(model,tokenizer,params):
        output_dir = 'Saved/bert'
#         if(params['train_att']):
#             if(params['lambda_attn']>=1):
#                 params['lambda_attn']=int(params['lambda_attn'])

#             output_dir =  output_dir+str(params['supervised_layer_pos'])+'_'+str(params['num_supervised_heads'])+'_'+str(params['num_classes'])+'_'+str(params['lambda_attn'])+'/'
            
#         else:
#             output_dir=output_dir+'_'+str(params['num_classes'])+'/'
#         print(output_dir)
#         # Create output directory if needed
#         if not os.path.exists(output_dir):
#             os.makedirs(output_dir)

#         print("Saving model to %s" % output_dir)

        # Save a trained model, configuration and tokenizer using `save_pretrained()`.
        # They can then be reloaded using `from_pretrained()`
        model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
        model_to_save.save_pretrained(output_dir)
        tokenizer.save_pretrained(output_dir)

In [32]:
save_bert_model(model,tokenizer,model.parameters())

In [33]:
torch.save(hate_space, 'hatespace.pt')
torch.save(hate_space_weights, 'hatespace.pt_weights')
torch.save(not_hate_space, 'not_hatespace.pt')
torch.save(not_hate_space_weights, 'not_hatespace_weights.pt')